In [106]:
import numpy as np
import pandas as pd
from google.cloud import bigquery
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import ttest_ind
import plotly.express as px
import db_dtypes
import bigframes.pandas as bpd
from IPython.display import display, HTML
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import math
import statsmodels.api as sm
from datetime import datetime
from typing import Union
import logging
import sys
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
from datetime import datetime, timedelta
import time



# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s %(levelname)s:%(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout)
    ]
)


mkt_data = """

WITH holdout_entities AS (
  SELECT
         entity_id,
        `Release Date` AS release_date
  FROM `logistics-data-storage-staging.long_term_pricing.global_holdout_rollout`
  WHERE `Release Date` < DATE_TRUNC(CURRENT_DATE(), WEEK)
  AND entity_id in ('PY_DO','PY_BO', 'FP_TW', 'PY_PY', 'DJ_CZ', 'PY_EC','MJM_AT' ,'PY_PE', 'PY_AR' ,'PY_GT','PY_SV' ,'FP_PH','PY_NI' ,'NP_HU' ,'FP_MM','EF_GR' ,'AP_PA'
  ,'YS_TR', 'PY_UY' ,'OP_SE' ,'PY_CL' ,'FP_BD' ,'FP_SG' ,'FO_NO' ,'PY_CR', 'FP_LA' ,'PO_FI','PY_HN', 'FP_MY' ,'FP_TH', 'FY_CY', 'PY_VE')
),
orders as (
  SELECT
      dps.entity_id entity_id
      ,dps.dps_customer_id customer_id
      ,COUNT(case when mkt.order_date <= e.release_date then mkt.order_id end) AS orders_pre
      ,COUNT(case when mkt.order_date > e.release_date then mkt.order_id end) AS orders_post
      ,SUM(case when mkt.order_date <= e.release_date then mkt.analytical_profit end) AS analytical_profit_pre
      ,SUM(case when mkt.order_date > e.release_date then mkt.analytical_profit end) AS analytical_profit_post
  FROM `fulfillment-dwh-production.cl.dps_sessions_mapped_to_orders` dps
  JOIN holdout_entities AS e
    ON dps.entity_id = e.entity_id
  left join fulfillment-dwh-production.curated_data_shared_mkt.bima_order_profitability mkt
    ON mkt.global_entity_id = dps.entity_id
    AND mkt.order_id = dps.platform_order_code
    AND order_date >= DATE_SUB(release_date, INTERVAL 8 WEEK)
    AND order_date < CURRENT_DATE
    AND global_entity_id in ('PY_DO','PY_BO', 'FP_TW', 'PY_PY', 'DJ_CZ', 'PY_EC','MJM_AT' ,'PY_PE', 'PY_AR' ,'PY_GT','PY_SV' ,'FP_PH','PY_NI' ,'NP_HU' ,'FP_MM','EF_GR' ,'AP_PA','YS_TR', 'PY_UY' ,'OP_SE' ,'PY_CL' ,'FP_BD' ,'FP_SG' ,'FO_NO' ,'PY_CR', 'FP_LA' ,'PO_FI','PY_HN', 'FP_MY' ,'FP_TH', 'FY_CY', 'PY_VE')
  WHERE dps.created_date BETWEEN DATE_SUB(e.release_date, INTERVAL 8 WEEK) AND CURRENT_DATE
    AND dps.platform_order_code IS NOT NULL
    AND dps.is_own_delivery
    AND dps.is_sent
    AND vendor_vertical_parent IN ('Restaurant','restaurant','restaurants')
    AND dps.entity_id in ('PY_DO','PY_BO', 'FP_TW', 'PY_PY', 'DJ_CZ', 'PY_EC','MJM_AT' ,'PY_PE', 'PY_AR' ,'PY_GT','PY_SV' ,'FP_PH','PY_NI' ,'NP_HU' ,'FP_MM','EF_GR' ,'AP_PA','YS_TR', 'PY_UY' ,'OP_SE' ,'PY_CL' ,'FP_BD' ,'FP_SG' ,'FO_NO' ,'PY_CR', 'FP_LA' ,'PO_FI','PY_HN', 'FP_MY' ,'FP_TH', 'FY_CY', 'PY_VE')
  GROUP BY 1, 2
), 
customer_information AS (
  SELECT
         e.entity_id,
         e.release_date,
         CASE WHEN d.created_date <= e.release_date THEN FALSE ELSE COALESCE(is_customer_holdout, FALSE) END AS is_customer_holdout,
         d.customer_id
  FROM `fulfillment-dwh-production.cl.dps_holdout_users` AS d
  JOIN holdout_entities AS e
        ON d.entity_id = e.entity_id
  LEFT JOIN `fulfillment-dwh-production.cl._bad_dps_logs_ids` bad_ids
    ON d.customer_id = bad_ids.id
  WHERE d.created_date BETWEEN '2025-01-01' AND CURRENT_DATE()
    AND d.customer_id IS NOT NULL
    AND bad_ids.id IS NULL
  GROUP BY 1, 2, 3, 4
)
SELECT 
   e.entity_id,
   e.customer_id,
   e.is_customer_holdout,
   o.orders_pre,
   o.orders_post,
   o.analytical_profit_pre,
   o.analytical_profit_post
FROM customer_information e
LEFT JOIN orders o
  ON o.customer_id = e.customer_id
  AND o.entity_id = e.entity_id

"""

dps_data = """

WITH holdout_entities AS (
  SELECT
         entity_id,
        `Release Date` AS release_date
  FROM `logistics-data-storage-staging.long_term_pricing.global_holdout_rollout`
  WHERE `Release Date` < DATE_TRUNC(CURRENT_DATE(), WEEK)
  AND entity_id not in ('PY_DO','PY_BO', 'FP_TW', 'PY_PY', 'DJ_CZ', 'PY_EC','MJM_AT' ,'PY_PE', 'PY_AR' ,'PY_GT','PY_SV' ,'FP_PH','PY_NI' ,'NP_HU' ,'FP_MM','EF_GR' ,'AP_PA'
  ,'YS_TR', 'PY_UY' ,'OP_SE' ,'PY_CL' ,'FP_BD' ,'FP_SG' ,'FO_NO' ,'PY_CR', 'FP_LA' ,'PO_FI','PY_HN', 'FP_MY' ,'FP_TH', 'FY_CY', 'PY_VE')
),
orders as (
  SELECT
      dps.entity_id entity_id
      ,dps.dps_customer_id customer_id
      ,COUNT(case when dps.created_date <= e.release_date then dps.platform_order_code end) AS orders_pre
      ,COUNT(case when dps.created_date > e.release_date then dps.platform_order_code end) AS orders_post
      ,SUM(case when dps.created_date <= e.release_date then dps.fully_loaded_gross_profit_eur end) AS analytical_profit_pre
      ,SUM(case when dps.created_date > e.release_date then dps.fully_loaded_gross_profit_eur end) AS analytical_profit_post
  FROM `fulfillment-dwh-production.cl.dps_sessions_mapped_to_orders` dps
  JOIN holdout_entities AS e
    ON dps.entity_id = e.entity_id
  WHERE dps.created_date BETWEEN DATE_SUB(e.release_date, INTERVAL 8 WEEK) AND CURRENT_DATE
    AND dps.platform_order_code IS NOT NULL
    AND dps.is_own_delivery
    AND dps.is_sent
    AND vendor_vertical_parent IN ('Restaurant','restaurant','restaurants')
    AND dps.entity_id not in ('PY_DO','PY_BO', 'FP_TW', 'PY_PY', 'DJ_CZ', 'PY_EC','MJM_AT' ,'PY_PE', 'PY_AR' ,'PY_GT','PY_SV' ,'FP_PH','PY_NI' ,'NP_HU' ,'FP_MM','EF_GR' ,'AP_PA','YS_TR', 'PY_UY' ,'OP_SE' ,'PY_CL' ,'FP_BD' ,'FP_SG' ,'FO_NO' ,'PY_CR', 'FP_LA' ,'PO_FI','PY_HN', 'FP_MY' ,'FP_TH', 'FY_CY', 'PY_VE')
  GROUP BY 1, 2
), 
customer_information AS (
  SELECT
         e.entity_id,
         e.release_date,
         CASE WHEN d.created_date <= e.release_date THEN FALSE ELSE COALESCE(is_customer_holdout, FALSE) END AS is_customer_holdout,
         d.customer_id
  FROM `fulfillment-dwh-production.cl.dps_holdout_users` AS d
  JOIN holdout_entities AS e
        ON d.entity_id = e.entity_id
  LEFT JOIN `fulfillment-dwh-production.cl._bad_dps_logs_ids` bad_ids
    ON d.customer_id = bad_ids.id
  WHERE d.created_date BETWEEN '2025-01-01' AND CURRENT_DATE()
    AND d.customer_id IS NOT NULL
    AND bad_ids.id IS NULL
  GROUP BY 1, 2, 3, 4
)
SELECT 
   e.entity_id,
   e.customer_id,
   e.is_customer_holdout,
   o.orders_pre,
   o.orders_post,
   o.analytical_profit_pre,
   o.analytical_profit_post
FROM customer_information e
LEFT JOIN orders o
  ON o.customer_id = e.customer_id
  AND o.entity_id = e.entity_id


"""

def combined_data(mkt, dps):

  mkt_df = client.query(mkt).to_dataframe()
  dps_df = client.query(dps).to_dataframe()

  # Append DataFrames
  combined = pd.concat([mkt_df, dps_df], ignore_index=True)

  return combined


#Function to create a dataset in Bigquery
def bq_create_dataset(client, dataset):
    dataset_ref = bigquery_client.dataset(dataset)

    try:
        dataset = bigquery_client.get_dataset(dataset_ref)
        print('Dataset {} already exists.'.format(dataset))
    except NotFound:
        dataset = bigquery.Dataset(dataset_ref)
        dataset.location = 'US'
        dataset = bigquery_client.create_dataset(dataset)
        print('Dataset {} created.'.format(dataset.dataset_id))
    return dataset

#Function to create a Table
def bq_create_table(client, dataset, table_name):
    dataset_ref = bigquery_client.dataset(dataset)

    table_ref = dataset_ref.table(table_name)

    try:
        table =  bigquery_client.get_table(table_ref)
        print('table {} already exists.'.format(table))
    except NotFound:
        schema = [
            bigquery.SchemaField('entity', 'STRING', mode='REQUIRED'),
            bigquery.SchemaField('sustainable_growth', 'FLOAT', mode='REQUIRED'),
            bigquery.SchemaField('percentage_change_orders_cuped', 'FLOAT', mode='REQUIRED'),
            bigquery.SchemaField('incremental_orders_cuped', 'FLOAT', mode='REQUIRED'),
            bigquery.SchemaField('non_holdout_total_orders_cuped', 'FLOAT', mode='REQUIRED'),
            bigquery.SchemaField('holdout_total_orders_cuped', 'FLOAT', mode='REQUIRED'),
            bigquery.SchemaField('t_stat_orders', 'FLOAT', mode='REQUIRED'),
            bigquery.SchemaField('p_value_orders', 'FLOAT', mode='REQUIRED'),
            bigquery.SchemaField('percentage_change_flgp_cuped', 'FLOAT', mode='REQUIRED'),
            bigquery.SchemaField('incremental_flgp_cuped', 'FLOAT', mode='REQUIRED'),
            bigquery.SchemaField('non_holdout_total_flgp_cuped', 'FLOAT', mode='REQUIRED'),
            bigquery.SchemaField('holdout_total_flgp_cuped', 'FLOAT', mode='REQUIRED'),
            bigquery.SchemaField('t_stat_flgp', 'FLOAT', mode='REQUIRED'),
            bigquery.SchemaField('p_value_flgp', 'FLOAT', mode='REQUIRED'),
            bigquery.SchemaField('holdout_flgp_per_order_cuped', 'FLOAT', mode='REQUIRED'),
            bigquery.SchemaField('non_holdout_flgp_per_order_cuped', 'FLOAT', mode='REQUIRED'),
            bigquery.SchemaField('holdout_orders_per_user_cuped', 'FLOAT', mode='REQUIRED'),
            bigquery.SchemaField('non_holdout_orders_per_user_cuped', 'FLOAT', mode='REQUIRED'),
            bigquery.SchemaField('metric_used', 'STRING', mode='REQUIRED'),
            bigquery.SchemaField('updated_date', 'DATE', mode='REQUIRED'),
        ]
        table = bigquery.Table(table_ref, schema=schema)
        table = bigquery_client.create_table(table)
        print('table {} created.'.format(table.table_id))
    return table

# Function to drop a table 
def drop_table(client, dataset_id, table_id):
    
    table_ref = client.dataset(dataset_id).table(table_id)
    client.delete_table(table_ref, not_found_ok=True)  # not_found_ok=True prevents errors if the table doesn't exist.
    print(f"Table {dataset_id}.{table_id} deleted successfully.")

# Function to insert rows to a table
def insert_df_rows_bigquery(client, dataset_id, table_id, df):
    
    # 1. Reference the table
    dataset_ref = client.dataset(dataset_id)
    table_ref = dataset_ref.table(table_id)

    # 2. Fetch the table object to get its schema/order
    table = client.get_table(table_ref)

    # 3. Convert the DataFrame to a list of dictionaries
    # Each dict corresponds to a row with column names as keys
    # These keys must match your table’s column names
    rows_to_insert = df.to_dict(orient='records')

    # 4. Insert the rows
    errors = client.insert_rows(table, rows_to_insert)

    # 5. Check for errors
    if errors:
        print("Encountered errors while inserting rows: ", errors)
    else:
        print(f"Successfully inserted {len(rows_to_insert)} rows into {dataset_id}.{table_id}.")
  

In [107]:
def calculate_sustainable_growth(data, pre_profit, post_profit):
    
    data = data.copy()
    entity = data['entity_id'].iloc[0]


    # Apply CUPED to FLGP
    data_flgp = data.dropna(subset=[pre_profit, post_profit])
    theta_flgp = np.cov(data_flgp[pre_profit], data_flgp[post_profit])[0, 1] / np.var(data_flgp[pre_profit])
    data_flgp['flgp_post_cuped'] = data_flgp[post_profit] - theta_flgp * (data_flgp[pre_profit] - data_flgp[pre_profit].mean())

    # Apply CUPED to Orders
    data_orders = data.dropna(subset=['orders_pre', 'orders_post'])
    theta_orders = np.cov(data_orders['orders_pre'], data_orders['orders_post'])[0, 1] / np.var(data_orders['orders_pre'])
    data_orders['orders_post_cuped'] = data_orders['orders_post'] - theta_orders * (data_orders['orders_pre'] - data_orders['orders_pre'].mean())

    # Per User Metrics
    holdout_flgpu_post = data_flgp.loc[data_flgp['is_customer_holdout'], 'flgp_post_cuped'].mean()
    holdout_flgpu_pre = data_flgp.loc[data_flgp['is_customer_holdout'], pre_profit].mean()
    
    non_holdout_flgpu_post = data_flgp.loc[~data_flgp['is_customer_holdout'], 'flgp_post_cuped'].mean()
    non_holdout_flgpu_pre = data_flgp.loc[~data_flgp['is_customer_holdout'], pre_profit].mean()

    holdout_orders_per_user_post = data_orders.loc[data_orders['is_customer_holdout'], 'orders_post_cuped'].mean()
    holdout_orders_per_user_pre = data_orders.loc[data_orders['is_customer_holdout'], 'orders_pre'].mean()
    
    non_holdout_orders_per_user_post = data_orders.loc[~data_orders['is_customer_holdout'], 'orders_post_cuped'].mean()
    non_holdout_orders_per_user_pre = data_orders.loc[~data_orders['is_customer_holdout'], 'orders_pre'].mean()

    # Apply DID
    # DID to flgpu
    d_flgpu_pre = non_holdout_flgpu_pre - holdout_flgpu_pre
    d_flgpu_post = non_holdout_flgpu_post - holdout_flgpu_post

    non_holdout_flgpu_adjusted_post = non_holdout_flgpu_post - d_flgpu_pre
    holdout_flgpu_adjusted_post = holdout_flgpu_post 

    did_flgpu  = non_holdout_flgpu_adjusted_post - holdout_flgpu_adjusted_post

    # DID to orders_per_user
    d_orders_per_user_pre = non_holdout_orders_per_user_pre - holdout_orders_per_user_pre
    d_orders_per_user_post = non_holdout_orders_per_user_post - holdout_orders_per_user_post

    non_holdout_orders_per_user_adjusted_post = non_holdout_orders_per_user_post - d_orders_per_user_pre
    holdout_orders_per_user_adjusted_post = holdout_orders_per_user_post

    did_orders_per_user  = non_holdout_orders_per_user_adjusted_post - holdout_orders_per_user_adjusted_post

    # Total FLGP and Orders
    holdout_user_count = data['is_customer_holdout'].sum()
    non_holdout_user_count = (~data['is_customer_holdout']).sum()

    holdout_total_flgp_cuped = holdout_flgpu_adjusted_post * holdout_user_count
    non_holdout_total_flgp_cuped = non_holdout_flgpu_adjusted_post * non_holdout_user_count

    holdout_total_orders_cuped = holdout_orders_per_user_adjusted_post * holdout_user_count
    non_holdout_total_orders_cuped = non_holdout_orders_per_user_adjusted_post * non_holdout_user_count

    # Normalize for Population Differences
    holdout_user_count = data['is_customer_holdout'].sum()
    non_holdout_user_count = (~data['is_customer_holdout']).sum()

    scaled_holdout_total_flgp_cuped = (holdout_total_flgp_cuped / holdout_user_count) * non_holdout_user_count if holdout_user_count != 0 else np.nan
    scaled_holdout_total_orders_cuped = (holdout_total_orders_cuped / holdout_user_count) * non_holdout_user_count if holdout_user_count != 0 else np.nan

    #Calculate Per order Metrics
    holdout_flgp_per_order_cuped = (
    np.nan if pd.isna(holdout_total_orders_cuped) or holdout_total_orders_cuped == 0
    else holdout_total_flgp_cuped / holdout_total_orders_cuped
    )
       
    non_holdout_flgp_per_order_cuped = (
    np.nan if pd.isna(non_holdout_total_orders_cuped) or non_holdout_total_orders_cuped == 0
    else non_holdout_total_flgp_cuped / non_holdout_total_orders_cuped
    )

    holdout_orders_per_user_cuped = holdout_orders_per_user_adjusted_post
    non_holdout_orders_per_user_cuped = non_holdout_orders_per_user_adjusted_post

    # Incremental Differences (CUPED)
    incremental_orders_cuped = non_holdout_total_orders_cuped - scaled_holdout_total_orders_cuped
    incremental_flgp_cuped = non_holdout_total_flgp_cuped - scaled_holdout_total_flgp_cuped

    # Percentage Changes (CUPED)
    percentage_change_orders_cuped = ((incremental_orders_cuped) / abs(scaled_holdout_total_orders_cuped)) * 100 if scaled_holdout_total_orders_cuped != 0 else np.nan
    percentage_change_flgp_cuped = ((incremental_flgp_cuped) / abs(scaled_holdout_total_flgp_cuped)) * 100 if scaled_holdout_total_flgp_cuped != 0 else np.nan

    # Sustainable Growth Calculation
    sustainable_growth = ((incremental_orders_cuped + (incremental_flgp_cuped / non_holdout_flgp_per_order_cuped)) / scaled_holdout_total_orders_cuped) * 100 if scaled_holdout_total_orders_cuped != 0 else np.nan

    # T-tests for significance
    t_stat_orders, p_value_orders = ttest_ind(
        data_orders.loc[data_orders['is_customer_holdout'], 'orders_post_cuped'],
        data_orders.loc[~data_orders['is_customer_holdout'], 'orders_post_cuped'],
        equal_var=False
    )

    t_stat_flgp, p_value_flgp = ttest_ind(
        data_flgp.loc[data_flgp['is_customer_holdout'], 'flgp_post_cuped'],
        data_flgp.loc[~data_flgp['is_customer_holdout'], 'flgp_post_cuped'],
        equal_var=False
    )

    return {
        'entity': entity,
        'sustainable_growth': sustainable_growth,
        'percentage_change_orders_cuped': percentage_change_orders_cuped,
        'incremental_orders_cuped': incremental_orders_cuped,
        'non_holdout_total_orders_cuped':non_holdout_total_orders_cuped,
        'holdout_total_orders_cuped':scaled_holdout_total_orders_cuped,
        't_stat_orders': t_stat_orders,
        'p_value_orders': p_value_orders,
        'percentage_change_flgp_cuped': percentage_change_flgp_cuped,
        'incremental_flgp_cuped': incremental_flgp_cuped,
        'non_holdout_total_flgp_cuped': non_holdout_total_flgp_cuped,
        'holdout_total_flgp_cuped':scaled_holdout_total_flgp_cuped,
        't_stat_flgp': t_stat_flgp,
        'p_value_flgp': p_value_flgp,
        'holdout_flgp_per_order_cuped': holdout_flgp_per_order_cuped,
        'non_holdout_flgp_per_order_cuped': non_holdout_flgp_per_order_cuped,
        'holdout_orders_per_user_cuped': holdout_orders_per_user_cuped,
        'non_holdout_orders_per_user_cuped': non_holdout_orders_per_user_cuped
    }



In [108]:
def extract_data(project , mkt_data, dps_data):
    
    project_id = project
    logging.info(f"Initializing BigQuery client for project: {project_id}")
    try:
        client = bigquery.Client(project=project_id)
    except Exception as e:
        logging.error(f"Failed to initialize BigQuery client: {e}")
        raise e
    
    start_time = time.time()
    combined_df = combined_data(mkt_data, dps_data)
    elapsed_time = time.time() - start_time
    logging.info(f"Time to extract and combine data from DB: {elapsed_time:.2f} seconds")
    
    return combined_df

def create_growth_dataframe(df):
    
    metric_pairs = [
        ('analytical_profit_pre', 'analytical_profit_post'),
    ]
    
    results = []
    start_time = time.time()
    for pre_metric, post_metric in metric_pairs:
        for entity in df['entity_id'].unique():
            entity_data = df[df['entity_id'] == entity]
            try:
                result = calculate_sustainable_growth(entity_data, pre_metric, post_metric)
                result['metric_used'] = f"{pre_metric}_vs_{post_metric}"
                results.append(result)
            except Exception as e:
                logging.error(f"Skipping entity {entity} due to error: {e}")
                continue
    elapsed_time = time.time() - start_time
    logging.info(f"Time to calculate sustainable growth: {elapsed_time:.2f} seconds")
    
    final_results_df = pd.DataFrame(results)
    final_results_df = final_results_df.dropna(subset=['sustainable_growth'])
    
    # Determine the current week's start (Monday)
    today = datetime.today().date()
    week_start = today - timedelta(days=today.weekday())
    final_results_df['updated_date'] = week_start
    
    csv_filename = f"profitable_growth_{week_start}.csv"
    final_results_df.to_csv(csv_filename, index=False)
    logging.info(f"CSV saved as {csv_filename}")
    
    return final_results_df

def push_data_to_bigquery(project,df, dataset="shazeb", table_name="abc_performance"):
    
    project_id = project
    try:
        bigquery_client = bigquery.Client(project=project_id_new)
    except Exception as e:
        logging.error(f"Failed to initialize BigQuery client for project {project_id_new}: {e}")
        raise e

    try:
        bq_create_dataset(bigquery_client, dataset)
        bq_create_table(bigquery_client, dataset, table_name)
    except Exception as e:
        logging.error(f"Error creating dataset/table: {e}")
        raise e
    
    try:
        insert_df_rows_bigquery(
            client=bigquery_client,
            dataset_id=dataset,
            table_id=table_name,
            df=final_results_df
        )
        logging.info("Data inserted successfully into BigQuery table.")
    except Exception as e:
        logging.error(f"Error inserting data into BigQuery: {e}")
        raise e

In [117]:
if __name__ == "__main__":

    # ------------------------------------------------------------------------------
    # GET DATA FROM DB
    # ------------------------------------------------------------------------------

    raw_data = extract_data("logistics-customer-staging",mkt_data, dps_data)
    
    # ------------------------------------------------------------------------------
    # Calculate Sustainable Growth
    # ------------------------------------------------------------------------------
    
    sustainable_df = create_growth_dataframe(raw_data)
    
    # ------------------------------------------------------------------------------
    # Push Data to DB
    # ------------------------------------------------------------------------------
    
    folder = "shazeb"
    table_name = "abc_performance"
    push_data_to_bigquery('logistics-data-storage-staging', sustainable_df, folder, table_name)


/Users/shazeb.asad/global_pricing/.venv/lib/python3.12/site-packages/google/auth/_default.py:76 UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds.

[2025-02-17T20:27:51.136+0100] {_default.py:683} WARNING - No project ID could be determined. Consider running `gcloud config set project` or setting the GOOGLE_CLOUD_PROJECT environment variable
Dataset Dataset(DatasetReference('logistics-data-storage-staging', 'shazeb')) already exists.
table logistics-data-storage-staging.shazeb.abc_performance already exists.
Successfully inserted 52 rows into shazeb.abc_performance.
[2025-02-17T20:27:52.565+0100] {2397728300.py:75} INFO - Data inserted successfully into BigQuery table.


In [114]:
# project_id = 'logistics-data-storage-staging'
# try:
#     bigquery_client = bigquery.Client(project=project_id)
# except Exception as e:
#     logging.error(f"Failed to initialize BigQuery client for project {project_id_new}: {e}")
#     raise e

# drop_table(bigquery_client, "shazeb", "abc_performance")

/Users/shazeb.asad/global_pricing/.venv/lib/python3.12/site-packages/google/auth/_default.py:76 UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds.

[2025-02-17T20:27:29.719+0100] {_default.py:683} WARNING - No project ID could be determined. Consider running `gcloud config set project` or setting the GOOGLE_CLOUD_PROJECT environment variable
Table shazeb.abc_performance deleted successfully.
